
## Homework Problem

Answer each of the following questions for an ideal cascade gaseous diffusion enrichment plant for two tail assay enrichment, 0.2 wt.% and 0.3 wt.% separately. You can ignore U losses during various process steps.

a) What are the required number of enriching and stripping sections? What is the total number of stages?  
b) How many SWUs are required?  
c) What is the energy consumption in producing the enriched material?  
d) How much does it cost (ore + conversion + enrichment) per year to produce the enriched material?

### Data
- One 1000 MWe PWR requires 22 MTU of 5.0 wt.% per year.
- Heads Separation Factor: $ \beta = 1.003 $
- Enrichment plant energy consumption feed material = 1.8 MWh/kg-SWU
- Feed material = 0.71 wt.%

### Cost estimate
- Ore: $ \$ 25 / \text{lb-U}_3\text{O}_8 $ (yellowcake)
- Conversion: $ \$ 8 / \text{kg-U} $ Produced as UF6
- Enrichment: $ \$ 50 / \text{SWU} $


In [49]:
from IPython.display import display, Math
import math

# Constants - enrichment has to be converted to molar fractions !!!
beta = 1.003  # Heads Separation Factor
x_p = 5.06 / 100  # Product enrichment (5.0 mol%)
x_t_1 = 0.203 / 100  # Tails enrichment 1 (0.2 mol%)
x_t_2 = 0.304 / 100  # Tails enrichment 2 (0.3 mol%)
x_f = 0.719 / 100  # Feed enrichment (0.71 mol%)
P = 22000  # Production amount in kg

# Calculate Abundance Ratios N_p (enriching stages) and N_w (stripping stages) for both tail enrichments
def calculate_stages(x_t):
    R_p = x_p / (1 - x_p)
    R_f = x_f / (1 - x_f)
    R_w = x_t / (1 - x_t)
    print(R_p, R_f, R_w)
    
    # Number of enriching sections Np
    N_p = math.log(R_p / R_f) / math.log(beta)
    
    # Number of stripping sections Nw
    N_w = math.log(R_f / R_w) / math.log(beta) - 1
    
    # Total number of stages
    N_total = N_p + N_w
    
    return N_p, N_w, N_total

# Calculate for both tail enrichments
N_p_1, N_w_1, N_total_1 = calculate_stages(x_t_1)
N_p_2, N_w_2, N_total_2 = calculate_stages(x_t_2)

N_p_1, N_w_1, N_total_1, N_p_2, N_w_2, N_total_2


# Display results
display(Math(r'\text{For the case in which the waste has enrichment }' + f'{x_t_1*100:.2f}' + r'\% \\ \text{Number of enriching stages: }' + f'{N_p_1:.0f}' + r'\text{ Number of stripping stages: }' + f'{N_w_1:.0f}' + r'\text{ Total number of stages: }' + f'{N_total_1:.0f}'))
display(Math(r'\text{For the case in which the waste has enrichment }' + f'{x_t_2*100:.2f}' + r'\% \\ \text{Number of enriching stages: }' + f'{N_p_2:.0f}' + r'\text{ Number of stripping stages: }' + f'{N_w_2:.0f}' + r'\text{ Total number of stages: }' + f'{N_total_2:.0f}'))


0.05329681904360649 0.007242070486800092 0.00203412928244336
0.05329681904360649 0.007242070486800092 0.0030492697801316


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [50]:
# Function for V(x)
def V(x):
    return (2 * x - 1) * math.log(x / (1 - x))
V_p = V(x_p)
V_f = V(x_f)

def SWU(x_t):
    W = P * ((x_p - x_f) / (x_f - x_t)) # U235 Conservation
    F = W + P # Total mass conservation
    return [P * V_p + W * V(x_t) - F * V_f, F]

SWU_1 = SWU(x_t_1)[0]
SWU_2 = SWU(x_t_2)[0]
SWU_1, SWU_2

# Energy consumption per SWU
energy_per_swu = 1.8  # MWh per SWU

# Calculate energy consumption for each case
energy_consumption_1 = SWU_1 * energy_per_swu
energy_consumption_2 = SWU_2 * energy_per_swu

energy_consumption_1, energy_consumption_2

# Display results
display(Math(r'\text{For the case in which the waste has enrichment }' + f'{x_t_1*100:.2f}' + r'\% \\ \text{SWU: }' + f'{SWU_1:.0f}' + r'\\ \text{ Energy consumption: }' + f'{energy_consumption_1:.0f}' + r'\text{ MWh}'))
display(Math(r'\text{For the case in which the waste has enrichment }' + f'{x_t_2*100:.2f}' + r'\% \\ \text{SWU: }' + f'{SWU_2:.0f}' + r'\\ \text{ Energy consumption: }' + f'{energy_consumption_2:.0f}' + r'\text{ MWh}'))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [51]:

# Constants
cost_ore_per_lb = 25  # $/lb for U3O8
conversion_cost_per_kg = 8  # $/kg for UF6 conversion
enrichment_cost_per_swu = 50  # $/SWU

# Convert ore cost to $/kg (1 kg = 2.20462 lb)
cost_ore_per_kg = cost_ore_per_lb * 2.20462

def cost(x_t):
    # Compute moles of Uranium
    moles_U = SWU(x_t)[1] / 238 # Approximate molar mass of U

    # Kg of U3O8
    kg_U3O8 = moles_U* (238*3 + 16*8)/3

    # Kg of UF6
    kg_UF6 = moles_U * (238 + 6*19)

    cost = kg_U3O8 * cost_ore_per_kg + kg_UF6 * conversion_cost_per_kg + SWU(x_t)[0] * enrichment_cost_per_swu
    return cost

tot_cost_1 = cost(x_t_1)
tot_cost_2 = cost(x_t_2)

# Display results
display(Math(r'\text{For the case in which the waste has enrichment }' + f'{x_t_1*100:.2f}' + r'\% \\ \text{Total cost: }' + f'{tot_cost_1*1e-6:.1f}' + r'\text{MLN USD}')) 
display(Math(r'\text{For the case in which the waste has enrichment }' + f'{x_t_2*100:.2f}' + r'\% \\ \text{Total cost: }' + f'{tot_cost_2*1e-6:.1f}' + r'\text{MLN USD}')) 



<IPython.core.display.Math object>

<IPython.core.display.Math object>